In [1]:
import csv

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
# To change comma decimal seperator in to dots
import locale as lc
lc.setlocale(lc.LC_ALL, 'nl_NL')

'nl_NL'

In [5]:
#set a path
path_raw_csv = 'C:\\Users\\Gebruiker\\Documents\\Personal_bookkeeping\\Raw_csv\\'

In [6]:
# Input the correct file.csv
# Don't forget to end your file with '.csv'
# Change month
month = "Nov20"
csv_ta_masterdata =pd.read_csv(path_raw_csv + 'NL69INGB0003880524_01-10-2020_31-10-2020.csv', sep= ';' , parse_dates = ["Date"] )


In [7]:
#csv_ta_masterdata

In [8]:
# select a subset of csv file
#(additional options are: Resulting balance, Notifications, Code  
csv_ta = csv_ta_masterdata.drop(columns=["Counterparty","Account","Resulting balance","Code", "Notifications", "Resulting balance", "Tag"])

#change column names
csv_ta = csv_ta.rename(columns={"Name / Description": "Description", "Amount (EUR)":"Amount"})


In [9]:
#changing Debit transactions from string to negative float type. 
#lc.atof is used to convert dutch ',' decimal seperator to '.' decimal seperator

for x in range(len(csv_ta["Amount"])): 
    if csv_ta.at[x,"Debit/credit"] == "Debit":
        csv_ta.at[x,"Amount"] = (lc.atof(csv_ta.at[x,"Amount"]) * (-1))
        
    elif csv_ta.at[x,"Debit/credit"] == "Credit":
        csv_ta.at[x,"Amount"] = (lc.atof(csv_ta.at[x,"Amount"]))
#upate table by dropping "Debit/credit" column
csv_ta = csv_ta.drop(columns ="Debit/credit") 

In [10]:
# add sorting_codes
sorting_codes = {}

sorting_codes["Supermarkets"]  = ["Westergracht", "ALBERT HEIJN", "Albert Heijn", "Vomar", "Aldi", "Jumbo", "Deka", "Raaks supermarkten"]

sorting_codes["Transport"]     = ["NSHeemstede", "NSHaarlem", "NSLeiden", "NSAmsterdam", "NS REIZIGERS"]

sorting_codes["Clothing"]      = []
                                                    #Groene Druif                               
sorting_codes["Horeca"]        = ["horeca","Cafe", "CCVJVG Horeca Beheer HEEMSTEDE", "Stiels"]
                                               #koffie HaarlemC
sorting_codes["Food & Drinks"] = ["MCDONALD", "CCVMAF"]

sorting_codes["Leisure etc."]  = ["Pathe", "Amazon"]

sorting_codes["Other"]         = ["BELASTINGDIEST", "Pearle", "OranjePakket", "CREDITCARD", "Body ? Fit", "Fit"]

sorting_codes["Monthly costs"] = ["UNICEF", "NSGK", "BUCKAROO"]

In [14]:
#create this column for unsorted TA's
csv_ta["Unsorted"] = 0

#assign transaction(amount) to column based on sorting codes    
for index in range(len(csv_ta["Date"])):
    #go through all the keywords in the library
    for key in sorting_codes:                              # [supermarkets, transport, leisure]
        csv_ta["XXX-" + key] = None                        # Make column for description
        csv_ta[key] = None                                 # Make column for amount

for index in range(len(csv_ta["Date"])): 
    for key in sorting_codes:
        for keyword in sorting_codes[key]:                                     # westergracht, albertheijn, vomar
            if keyword in csv_ta.at[index,"Description"]:                      # If keyword in in the description
                csv_ta.at[index,"XXX-" + key] = csv_ta.at[index,"Description"]   # Add description 
                csv_ta.at[index,key] = csv_ta.at[index,"Amount"]                 # add amount
            else:  
                csv_ta.at[index,"XXX-Other"] = csv_ta.at[index,"Description"]   # Add description 
                csv_ta.at[index,"Other"] = csv_ta.at[index,"Amount"]
                csv_ta.at[index,"Unsorted"] = 1
            
#for key in sorting_codes:
#    csv_ta.rename(columns={"XXX-"+key : "x"})
        
        

# Mark all unsorted transactions for further inspection
# csv_ta["Sorted"] = 0
#for index in range(len(csv_ta["Date"])):
#    for key in sorting_codes:
#        if csv_ta.at[index,key] != None:
#            csv_ta.at[index,"Sorted"] = 1
#        else:
#            pass
  

In [15]:
csv_ta

,Date,Description,Amount,Transaction type,Unsorted,XXX-Supermarkets,Supermarkets,XXX-Transport,Transport,XXX-Clothing,...,XXX-Horeca,Horeca,XXX-Food & Drinks,Food & Drinks,XXX-Leisure etc.,Leisure etc.,XXX-Other,Other,XXX-Monthly costs,Monthly costs
0,2020-10-30,Westergracht Supermark HAARLEM,-29.06,Payment terminal,1,Westergracht Supermark HAARLEM,-29.06,None,None,None,...,None,None,None,None,None,None,Westergracht Supermark HAARLEM,-29.06,None,None
1,2020-10-30,Gall Gall Haarlem HAARLEM NLD,-3.29,Payment terminal,1,None,None,None,None,None,...,None,None,None,None,None,None,Gall Gall Haarlem HAARLEM NLD,-3.29,None,None
2,2020-10-28,STICHTING DERDENGELDEN BUCKAROO,-6,SEPA direct debit,1,None,None,None,None,None,...,None,None,None,None,None,None,STICHTING DERDENGELDEN BUCKAROO,-6,STICHTING DERDENGELDEN BUCKAROO,-6
3,2020-10-27,NSGK-Het Gehandicapte Ki,-7,SEPA direct debit,1,None,None,None,None,None,...,None,None,None,None,None,None,NSGK-Het Gehandicapte Ki,-7,NSGK-Het Gehandicapte Ki,-7
4,2020-10-27,WILLIE WORTEL SIN HAARLEM NLD,-29,Payment terminal,1,None,None,None,None,None,...,None,None,None,None,None,None,WILLIE WORTEL SIN HAARLEM NLD,-29,None,None
5,2020-10-22,NS GROEP IZ NS REIZIGERS,-3.85,SEPA direct debit,1,None,None,NS GROEP IZ NS REIZIGERS,-3.85,None,...,None,None,None,None,None,None,NS GROEP IZ NS REIZIGERS,-3.85,None,None
6,2020-10-21,Albert Heijn 1463 HEEMSTEDE NLD,-14.5,Payment terminal,1,Albert Heijn 1463 HEEMSTEDE NLD,-14.5,None,None,None,...,None,None,None,None,None,None,Albert Heijn 1463 HEEMSTEDE NLD,-14.5,None,None
7,2020-10-20,D M E NESSEN BURMANJE,-244.47,Online Banking,1,None,None,None,None,None,...,None,None,None,None,None,None,D M E NESSEN BURMANJE,-244.47,None,None
8,2020-10-20,D M E NESSEN BURMANJE,244.47,Transfer,1,None,None,None,None,None,...,None,None,None,None,None,None,D M E NESSEN BURMANJE,244.47,None,None
9,2020-10-20,ALBERT HEIJN 1557 LEIDEN NLD,-11.51,Payment terminal,1,ALBERT HEIJN 1557 LEIDEN NLD,-11.51,None,None,None,...,None,None,None,None,None,None,ALBERT HEIJN 1557 LEIDEN NLD,-11.51,None,None


In [117]:
x = 4
for key in sorting_codes:
    key = round(total[x], 2)
    x =+ 1

In [118]:
Supermarkets

NameError: name 'Supermarkets' is not defined

In [112]:
# creating Summary Statistics
# Sum all Axis
total = csv_ta.sum(axis=0)

# connecting code to stat
Netto = round(total[1], 2)

x = 4
for key in sorting_codes:
    key = round(total[x], 2)
    x =+ 1
    
#Put stats in dict
totals_dic = {month : pd.Series([Netto, Supermarkets, Transport, Clothing , Coffee_shops, Horeca, Food_n_Drinks, Leisure_etc, Other, Monthly_costs], 
                     index=['Netto','Boodschappen','OV','Kleding','Coffeeshops','Drankjes','Eten & Drinken','Vrije Tijd', "Other", "Montly_costs"])}
#data frame dict
Month_totals = pd.DataFrame(totals_dic)    
    

SyntaxError: positional argument follows keyword argument (<ipython-input-112-a00d74bc2f70>, line 3)

In [ ]:
csv_ta.sum(axis=0)

In [16]:
#export to Bookkeeping Excel file
with pd.ExcelWriter('bookkeeping_overview.xlsx',engine="openpyxl",  mode='a') as writer:
    csv_ta.to_excel(writer,month,index=False)
    
#export / append totals overview to Excel File    
#Month_totals.to_csv(month+'_totals.csv')

In [13]:
csv_ta

,Date,Description,Amount,Transaction type
0,2020-10-30,Westergracht Supermark HAARLEM,-29.06,Payment terminal
1,2020-10-30,Gall Gall Haarlem HAARLEM NLD,-3.29,Payment terminal
2,2020-10-28,STICHTING DERDENGELDEN BUCKAROO,-6,SEPA direct debit
3,2020-10-27,NSGK-Het Gehandicapte Ki,-7,SEPA direct debit
4,2020-10-27,WILLIE WORTEL SIN HAARLEM NLD,-29,Payment terminal
5,2020-10-22,NS GROEP IZ NS REIZIGERS,-3.85,SEPA direct debit
6,2020-10-21,Albert Heijn 1463 HEEMSTEDE NLD,-14.5,Payment terminal
7,2020-10-20,D M E NESSEN BURMANJE,-244.47,Online Banking
8,2020-10-20,D M E NESSEN BURMANJE,244.47,Transfer
9,2020-10-20,ALBERT HEIJN 1557 LEIDEN NLD,-11.51,Payment terminal
